# Patrones creacionales

Source1: https://refactoring.guru/design-patterns/creational-patterns
Source2

De acuerdo a la fuente anterior:
*Creational design patterns provide various object creation mechanisms, which increase flexibility and reuse of existing code*

Tenemos los siguientes tipos:

- Factory Method
- Singleton
- Abstract Factory
- Builder
- Prototype


## Singleton

*Singleton is a creational design pattern that lets you ensure that a class has only one instance, while providing a global access point to this instance.*

*It is a way to provide one and only one object of a particular type. It involves only one class to create methods and specify the objects.*

**Definition:** The singleton pattern is a design pattern that restricts the instantiation of a class to one object.

Este patron resuelve 2 problemas 

1. Asegurarse que una clase tenga una sola instancia
Cada vez que quisieramos crear un objeto, en lugar de hacerlo, se nos da el objeto previamente creado. Es decir accedemos siempre a la misma instancia.
Este tipo de patron es incopatible con un constructor ya que el constructor siempre nos retorna un nuevo objeto.

2. Provee un punto de acceso global a dicha instancia.

A pesar de poder acceder a este objeto de forma global, el mismo esta protegido contra sobre-escritura.

### Creacion de un singleton en python
Para craer un singleto tenemos que implementar los siguientes pasos:

- Hacer que el constructor default sea privado. Esto previene que otros objetos utilizen el operador new
- Crear un metodo de creacion estatico que actue como constructor

En este post se discuten ciertas manera de crear un singleton pero de momento me superan:
https://stackoverflow.com/questions/6760685/creating-a-singleton-in-python

Source3: https://www.pythonmorsels.com/making-singletons/
Esta fuente es buenisima y explica la creacion de un singleton paso a paso y explicando todo lo que necesitas saber para entender.

Supongamos que tenemos una clase, y quisieramos que fuera singleton:

In [5]:
#Esta es mi clase singleton
class pokemon:
    
    def __init__(self,name,attack,defense,hp):
        self.name = name
        self.attack = attack
        self.defense = defense
        self.hp = hp

    def __repr__(self):
        return str({'name':self.name,'attack':self.attack,'defense':self.defense,'hp':self.hp})

pikachu1 = pokemon('pikachu',123,55,500)
pikachu2 = pokemon('pikachu',123,55,500)
print(pikachu1 == pikachu2)
print(id(pikachu1),id(pikachu2))

False
2022399968592 2022399981968


Lo que se puede ver aqui es que nuestra clase, es una clase normal y los objetos que entrega son distintos. Si esto fuera un singleton tendriamos objetos iguales.

In [1]:
#A partir de esta clase creemos el singleton

class pokemon:
    _self = None #En principio esto no existe
    def __new__(cls,name,attack,defense,hp):
        if cls._self is None:
            cls._self = super().__new__(cls)
        return cls._self
    def __init__(self,name,attack,defense,hp):
        self.name = name
        self.attack = attack
        self.defense = defense
        self.hp = hp
    def __repr__(self):
        return str({'name':self.name,'attack':self.attack,'defense':self.defense,'hp':self.hp})
    
pikachu1 = pokemon('pikachu',123,55,500)
pikachu2 = pokemon('pikachu',123,55,500)
print(pikachu1 == pikachu2)
print(id(pikachu1),id(pikachu2))

True
1968692585616 1968692585616


Como puede verse finalmente ahora las dos instancias son las mismas.

## Factory Method

<a href='https://realpython.com/factory-method-python/'>Source</a>

Es un patron de disenio creacional que propone una interfaz para crear objetos en una superclase, mientras permite a las subclases alterar el tipo de objetos que se crea.

- Factory Method es un patron que crea objetos a partir de una interfaz comun. *Factory Method is a creational design pattern used to create concrete implementations of a common interface.*

- Separa el proceso de crear un objeto del codigo que depende de la interfaz del objeto

En el articulo de Real Python encontraremos el siguiente codigo de un serializador de canciones:

In [ ]:
import json
import xml.etree.ElementTree as et

class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist


class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)
        elif format == 'XML':
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError(format)

Lo que tenemos aca son basicamente 2 cosas:

1. La clase song que sirve para describir objetos cancion, utilizando el id, title y artis
2. La clase song serializer que, con un conjunto de if permite seleccionar un tipo de archivo y transformarlo

Entonces para hacer funcionar primero tenemos que crear un objeto clase song, y despues utilizar el serialize con este objeto song y especificando un formato.

El codigo anterior tiene 2 desventajas , la primera es una complejidad de entendimiento debido a la cadena de if/elif/else y con mayor importancia viola el *single responsability principle* donde se indica que cualquier modulo, clase o inclusive un metodo de clase debe tener una responsabilidad unica bien definida.

Otros problemas son:

- Cuando quiera introducir un nuevo formato, hay que retocar la clase para sostenerlo
- Si las propiedades del objeto song cambian hay que ir a tocar codigo del song serializer

La siguiente pregunta es, que cambios necesita el codigo?

El codigo que utiliza if/else/elif usualmente tiene un objetivo comun. Cada camino, busca convertir el objeto song a un formato.
Basados en este objetivo buscamos elaborar una interfaz comun la cual provea implementaciones separadas para cada camino del if.
Despues deberiamos agregar un componente separado que decida cual implementacion utilizar basado en el formato.

Para realizar todas estas labores tenemos que pensar en una **REFACTORIZACION** del codigo: *the process of changing a software system in such a way that does not alter the external behavior of the code yet improves its internal structure*

Esta refactorizacion va a consistir en lo siguiente:

In [ ]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_json(song)
        elif format == 'XML':
            return self._serialize_to_xml(song)
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

SongSerializer es unuestra interfaz comun que basada en un formato hace funcionar un metodo de convercion especifico para ese formato.

### Ahora si Factory method
El objetivo del Factory Method es el de proveer un componente separado con la responsabilidad de decidir cual implementacion concreta debe ser utilizada basandose en un parametro especificado.

Entonces lo que vamos a hacer es agregar una clase extra a la cual le delegaremos la responsabilidad de decidir que metodo usar basandonos en el formato dado en input por el usuario:

In [ ]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = self._get_serializer(format)
        return serializer(song)

    def _get_serializer(self, format):
        if format == 'JSON':
            return self._serialize_to_json
        elif format == 'XML':
            return self._serialize_to_xml
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

serialize recibe los mismos input pero ahora en lugar de utilizar directamente el bloque if, simplemente llama a otro metodo que fue recientemente creado: _get_serializer el cual tiene la labor de decidir cual es el metodo que vamos a utlizar para serializar nuestro objeto song. Observar que este metodo no recibe song, simplemente el formato. Lo que retorna es el metodo que vamos a utilizar.

Es basicamente el tipo que le das una tarjetita con el formato y va a buscar a la bodega la maquinita especifica que te hace la transformacion.

Ahora demosle nombres a estas cosas:

- .serialize() es el metodo de la aplicacion que depende de una *interfaz* para realizar sus tareas. Este es referido como el **client component** de la aplicacion

- El conjunto, SongSerializer es la interfaz y se denomina como **product component** . En nuestro caso, el producto es una funcion que recibe una cancion y entrega una representacion string

- _serialize_to_json y _serialize_to_xml son implementaciones concretas del producto.

- _get_serializer es el componente creador. Este componente decide, como dijimos, que implementacion utilizar

Como cuestion final, como ninguno de los metodos utiliza el parametro self ralmente, entonces se separa las mismas en funciones en lugar de pertenecer a la misma clase (ver articulo) que esten dentro de la misma clase no es indispensable para el patron.

Los siguientes parrafos del articulo son iluminadores:

*The mechanics of Factory Method are always the same. A client (SongSerializer.serialize()) depends on a concrete implementation of an interface. It requests the implementation from a creator component (get_serializer()) using some sort of identifier (format).*

*The creator returns the concrete implementation according to the value of the parameter to the client, and the client uses the provided object to complete its task.*

Veamos un pequenio ejemplo de aplicacion:



In [3]:
def suma(a,b):
    return a+b
def resta(a,b):
    return a-b


def method_finder(user_op):
    operaciones = ['suma','resta']
    metodos = [suma,resta]
    library = dict(zip(operaciones,metodos))
    if user_op not in library:
        raise ValueError('The operation is not in the library')
    return library[user_op]

def user_interface(a,b,user_op):
    operation = method_finder(user_op)
    return operation(a,b)

print(user_interface(3,3,'resta'))
print(user_interface(3,3,'divition'))

0


ValueError: The operation is not in the library

In [ ]:
#Otro ejemplo mas
from .io import read_table

# Interfaz del usuario 
def void_builder(path,void_type):
    box = read_table(path)
    method = get_builder(void_type)
    return method(box)
    
# componente creador
def get_builder(void_type):
    finders = ['spherical','popcorn']
    methods = [spherical_void_finder,popcorn_void_finder]
    library = dict(zip(finders,methods))
    if void_type not in library:
        raise ValueError('The method is not in the library')
    return library[void_type]
    
def spherical_void_finder(box):
    spherical_void = 'Aca uso el algoritmo de Andres con el box'
    return spherical_void

def popcorn_void_finder(box):
    popcorn_void = 'Aca uso el algoritmo de Dante con el box'
    return popcorn_void